In [ ]:
import pandas as pd
import regex as re
import nltk
from nltk.corpus import stopwords
from collections import Counter
from collections import defaultdict
import math

In [ ]:
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

In [ ]:
def read_tsv_file(path_file):
    sentences = list()
    sentence = list()
    with open(path_file) as file:
        for line in file:
            if line != "\n":
                lemma = line.split('\t')[2]
                sentence.append(lemma[:-1])
            if line == "\n" and sentence:
                sentences.append(sentence)
                sentence = list()
    return sentences


In [ ]:
def remove_punct_marks(data):
    clear_data = list()
    for sentence in data:
        sentence_data = [re.sub(r"[^\P{P}-]+","", lemma.lower()) for lemma in sentence if re.sub(r"[^\P{P}-]+","", lemma.lower()) and lemma.lower() not in stop_words]
        clear_data.append(sentence_data)
    return clear_data

In [ ]:
def extract_trigrams(clear_data):
    trigrams = list()
    for sentence in clear_data:
        for i in range(len(sentence) - 2):
            trigrams.append(sentence[i:i+3])
    return trigrams


In [ ]:
def count_words(clear_data):
    f_words = dict()
    for sentence in clear_data:
        for word in sentence:
            if word not in f_words:
                f_words[word] = 1
            else:
                f_words[word] += 1
    return f_words

In [ ]:
def t_score(trigram_frequencies, word_counts, total_words):
    t_scores = {}
    for trigram, count in trigram_frequencies.items():
        expected_frequency = (word_counts[trigram[0]] / total_words) * \
                             (word_counts[trigram[1]] / total_words) * \
                             (word_counts[trigram[2]] / total_words) * total_words
        t_score = (count - expected_frequency) / math.sqrt(count)
        t_scores[trigram] = t_score
    return t_scores

In [ ]:
def trigram_frequencies(trigrams_data):
    trigram_frequency = dict()
    for trigram in trigrams_data:
        if tuple(trigram) not in trigram_frequency:
            trigram_frequency[tuple(trigram)] = 1
        else:
            trigram_frequency[tuple(trigram)] += 1
    return trigram_frequency

In [ ]:
data = read_tsv_file('/Users/ddhearted/Desktop/NLP/lab1/train/rec.autos/101555.tsv')
clear_data = remove_punct_marks(data)

trigrams_data = extract_trigrams(clear_data)
word_counts = count_words(clear_data)

trigram_frequency = trigram_frequencies(trigrams_data)
total_words = sum(word_counts.values())

t_scores = t_score(trigram_frequency, word_counts, total_words)

sorted_t_scores = sorted(t_scores.items(), key=lambda x: x[1], reverse=True)[:10]

for trigram, score in sorted_t_scores:
    print(f"{trigram}: {score}")

In [ ]:
import nltk
from nltk.collocations import *
from nltk.corpus import PlaintextCorpusReader

bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

f = open('/Users/ddhearted/Desktop/NLP/lab1/train/rec.autos/101555.tsv')
raw = f.read()

tokens = nltk.word_tokenize(raw,'english',True)
print(tokens[:60])

In [ ]:
trigram_measures = nltk.collocations.TrigramAssocMeasures()

flat_list = [item for sublist in clear_data for item in sublist]
finder_thr = TrigramCollocationFinder.from_words(flat_list)

In [ ]:
finder_thr.nbest(trigram_measures.student_t, 10)